In [65]:
# Dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.config import load_config_nb

sns.set('notebook', font_scale=1.1, rc={'figure.figsize': (10, 3)})
sns.set_style('ticks', rc={'figure.facecolor': 'none', 'axes.facecolor': 'none'})
%config InlineBackend.figure_format = 'svg'

import yaml
from nocturne.envs.base_env import BaseEnv

### Settings

In [66]:
env_config = load_config_nb("env_config")

env_config.data_path = "../data_lp/"
env_config.subscriber.use_observations = False
env_config.subscriber.use_ego_state = False
env_config.subscriber.use_current_position = True
env_config.dt = 0.1 # Default is 0.1
env_config.normalize_state = False

# Action space
env_config.accel_discretization = 1
env_config.steering_discretization = 3
env_config.accel_lower_bound = 0.0
env_config.accel_upper_bound = 0.0
env_config.steering_lower_bound = -0.5
env_config.steering_upper_bound = 0.5

In [67]:
# Make environment
env = BaseEnv(env_config)

### Linear Programming

In [74]:
# Make environment
env = BaseEnv(env_config)
env.reset()

{3: array([ 9.03282031e+03, -2.71831323e+03,  1.10296458e-01,  0.00000000e+00])}

In [139]:
from itertools import product

# Constants
NUM_REP_ACTION = 1
AGENT_ID = 3
VERBOSE = False

# Reset environment and get initial state
init_state = env.reset()[AGENT_ID]

# Get maximum number of actions
max_actions = 8
print(f"Max actions: {max_actions}")


# Define number of states, reward, transition matrix, and discount factor
num_actions = env.action_space.n
num_states = num_actions ** max_actions
states = {}
reward = {}
transition = {}
gamma = 1.0

# Set initial state and reward
states[0] = init_state
reward[0] = 0

# Get all possible action sequences
action_seqs = list(product(env.idx_to_actions, repeat=max_actions))

# Iterate through all action sequences
state_idx = 1
for action_seq_idx, action_seq in enumerate(action_seqs):

    # Print progress
    if VERBOSE:
        print(f"Action index: {action_seq_idx + 1} / {len(action_seqs)}")
        print(f"Action sequence: {action_seq}")
    else:
        print("\rAction index: {} / {}".format(action_seq_idx + 1, len(action_seqs)), end="")
    
    # Reset environment
    obs = env.reset()
    
    # Step through scene using action sequence
    for action_idx, action in enumerate(action_seq):
        
        # Reset reward for action
        total_reward = 0

        # Repeat action
        for _ in range(NUM_REP_ACTION):
            obs, rew, done, info = env.step({AGENT_ID: action})
            total_reward += rew[AGENT_ID]
            if done['__all__']:  # Stop if agent is done
                total_reward += 10 if info[AGENT_ID]['goal_achieved'] else -10
                if VERBOSE:
                    print("GOAL ACHIEVED!" if info[AGENT_ID]['goal_achieved'] else "Oh no, the car crashed!")
                break
        
        # Get next state
        next_state = obs[AGENT_ID]

        # Store state, reward, and transition
        states[state_idx] = next_state
        reward[state_idx] = total_reward
        transition[((state_idx - 1) if (action_idx != 0) else 0, action)] = state_idx

        # Increment state index
        state_idx += 1

        if done['__all__']:  # Stop if agent is done
            break
    if not done['__all__'] and VERBOSE:
        print("The car is not done driving...")
    
    if VERBOSE:
        print("\n")

if not VERBOSE:
    print("\n")

# Remove duplicate states
states_df = pd.DataFrame(
    index = states.keys(),
    data = states.values(),
    columns = ["x", "y", "speed", "angle"],
)
states_full_df = pd.merge(
    left = states_df,
    right = states_df.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={"index": "state_map"}),
    how = "left",
    on = ["x", "y", "speed", "angle"],
)
state_map = states_full_df["state_map"].to_dict()

# Remap states, rewards, and transitions
states = {state_map[state_idx]: state for state_idx, state in states.items()}
reward = {state_map[state_idx]: rew for state_idx, rew in reward.items()}
transition = {(state_map[state_idx], action): state_map[next_state_idx] for (state_idx, action), next_state_idx in transition.items()}

# Convert transition to 3D numpy array
transition_array = np.zeros((len(states), num_actions, len(states)))
for (state_idx, action), next_state_idx in transition.items():
    transition_array[state_idx, action, next_state_idx] = 1

# Display number of states
print(f"Number of states: {len(states)}")

Max actions: 8
Action index: 6561 / 6561

Number of states: 649


In [140]:
try:
    from pyomo.environ import ConcreteModel, Var, minimize, SolverFactory
except ModuleNotFoundError:
    !pip install pyomo
    from pyomo.environ import ConcreteModel, Var, minimize, SolverFactory

# Step 0: Create an instance of the model
model = ConcreteModel()

# Step 1: Define index sets
states_set = list(range(len(states)))
actions_set = list(range(num_actions))

# Step 2: Define the decision 
model.v = Var(states_set, initialize=0)

# Step 3: Define Objective
@model.Objective(sense=minimize)
def objective(m):
    return sum([model.v[s] for s in states_set])

# Step 4: Constraints
@model.Constraint(states_set, actions_set)
def state_value_constraint(m, s, a):
    return model.v[s] >= reward[s] + gamma * sum([transition_array[s, a, s2] * model.v[s2] for s2 in states_set])

# Step 5: Solve
# results = SolverFactory('cbc').solve(model)
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 30.9605320436885
  Upper bound: 30.9605320436885
  Number of objectives: 1
  Number of constraints: 1947
  Number of variables: 649
  Number of nonzeros: 3201
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.2935659885406494
# ----------------------------------------------------------
#   Solution Information
# --------

In [141]:
# Display results
model.display()

Model unknown

  Variables:
    v : Size=649, Index=v_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :  None :  0.155271743154448 :  None : False : False :  Reals
          1 :  None :  0.155271743154448 :  None : False : False :  Reals
          2 :  None :  0.136042630248376 :  None : False : False :  Reals
          3 :  None :  0.116762024255567 :  None : False : False :  Reals
          4 :  None : 0.0974301632164545 :  None : False : False :  Reals
          5 :  None : 0.0780467394750054 :  None : False : False :  Reals
          6 :  None : 0.0586121347941823 :  None : False : False :  Reals
          7 :  None : 0.0391260841855755 :  None : False : False :  Reals
          8 :  None : 0.0195886999324094 :  None : False : False :  Reals
          9 :  None : 0.0195888032329759 :  None : False : False :  Reals
         10 :  None : 0.0195861555945431 :  None : False : False :  Reals
         11 :  None : 0.0391262773127215 :  None : F

In [142]:
# Get LP solution values
values = np.array([val.value for val in model.v.values()])

In [143]:
# Find optimal path and actions
opt_states = [0]
opt_actions = []
for _ in range(15):
    next_values = np.multiply(transition_array[opt_states[-1], :, :], values)
    if (next_values == 0).all():
        break
    opt_next = next_values.max(axis=0).argmax()
    opt_action = transition_array[opt_states[-1], :, opt_next].argmax()
    opt_states.append(opt_next)
    opt_actions.append(opt_action)

# Display optimal path and actions
pd.DataFrame(
    index = opt_states,
    data = {
        "x": [states[state_idx][0] for state_idx in opt_states],
        "y": [states[state_idx][1] for state_idx in opt_states],
        "speed": [states[state_idx][2] for state_idx in opt_states],
        "angle": [states[state_idx][3] for state_idx in opt_states],
        "value": [values[state_idx] for state_idx in opt_states],
        "action": [*[opt_action for opt_action in opt_actions], None],
    },
).style.set_caption("<h4>Optimal Path</h4>")

,x,y,speed,angle,value,action
0,9032.820312,-2718.313232,0.110296,0.000000,0.155272,0.000000
1,9032.825195,-2718.323242,0.110296,-0.500000,0.155272,1.000000
213,9032.832031,-2718.331787,0.110296,0.000000,0.136043,1.000000
215,9032.838867,-2718.340332,0.110296,0.000000,0.116763,1.000000
217,9032.845703,-2718.348877,0.110296,0.000000,0.097431,1.000000
219,9032.852539,-2718.357422,0.110296,0.000000,0.078048,1.000000
221,9032.859375,-2718.365967,0.110296,0.000000,0.058613,1.000000
223,9032.866211,-2718.374512,0.110296,0.000000,0.039127,1.000000
225,9032.873047,-2718.383057,0.110296,0.000000,0.019589,nan
